<hr><center><img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks&para;&para;&para;">unSkript Runbooks</h1>
<div class="alert alert-block alert-success">
<h3 id="Objective&para;&para;&para;">Objective</h3>
<br><strong><em>Stop untagged EC2 Instances</em></strong></div>
</center>
<p>&nbsp;</p>
<center>
<h2 id="Restart-Unhealthy-Services-in-Target-Group&para;&para;&para;"><u>Restart Unhealthy Services in Target Group</u></h2>
</center>
<h1 id="Steps-Overview&para;&para;&para;">Steps Overview</h1>
<p>1. List Unhealthy Instances in a Target Group<br>2. Restart EC2 instances</p>
<hr>

In [21]:
if instance_ids and not region:
    raise SystemExit("Provide region for the instance!")
if region == None:
    region = ""

<h3 id="List-Unhealthy-Instances-in-a-Target-Group&para;"><a id="2" target="_self" rel="nofollow"></a>List Unhealthy Instances in a Target Group</h3>
<p>Here we will fetch all the untagged&nbsp; EC2 instances.&nbsp;</p>
<blockquote>
<p>This action takes the following parameters: <code>region(Optional)</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>unheathy_instances</code></p>
</blockquote>

In [ ]:
import pprint
from typing import Optional, Tuple
from pydantic import BaseModel, Field
from unskript.connectors.aws import aws_get_paginator
from unskript.legos.aws.aws_list_all_regions.aws_list_all_regions import aws_list_all_regions
from unskript.legos.utils import parseARN

from beartype import beartype
@beartype
def aws_list_unhealthy_instances_in_target_group_printer(output):
    if output is None:
        return
    pprint.pprint(output)

@beartype
def get_all_target_groups(handle, r):
    target_arns_list = []
    elbv2Client = handle.client('elbv2', region_name=r)
    try:
        tbs = aws_get_paginator(elbv2Client, "describe_target_groups", "TargetGroups")
        for index, tb in enumerate(tbs):
            target_arns_list.append(tb.get('TargetGroupArn'))
    except Exception:
        pass
    return target_arns_list

@beartype
def aws_list_unhealthy_instances_in_target_group(handle, region: str=None) -> Tuple:
    result = []
    unhealthy_instances_list = []
    all_target_groups = []
    unhealhthy_instances_dict ={}
    all_regions = [region]
    if region is None or len(region)==0:
        all_regions = aws_list_all_regions(handle=handle)
    for r in all_regions:
        try:
            output = get_all_target_groups(handle,r)
            if len(output)!=0:
                all_target_groups.append(output)
        except Exception as e:
            print("????????")
            pass
    for target_group in all_target_groups:
        for o in target_group:
            parsedArn = parseARN(o)
            region_name = parsedArn['region']
            elbv2Client = handle.client('elbv2', region_name=region_name)
            try:
                targetHealthResponse = elbv2Client.describe_target_health(TargetGroupArn=o)
            except Exception as e:
                raise e
            for ins in targetHealthResponse["TargetHealthDescriptions"]:
                if ins['TargetHealth']['State'] in ['unhealthy']:
                    unhealthy_instances_list.append(ins['Target']['Id'])
    if len(unhealthy_instances_list)!=0:
        unhealhthy_instances_dict['instance'] = unhealthy_instances_list
        unhealhthy_instances_dict['region'] = region_name
        result.append(unhealhthy_instances_dict)
    if len(result)!=0:
        return (False,result)
    return (True, None)


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "region": "region"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not instance_ids",
    "condition_result": true
    }''')

task.configure(outputName="unhealthy_instances")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_list_unhealthy_instances_in_target_group, lego_printer=aws_list_unhealthy_instances_in_target_group_printer, hdl=hdl, args=args)

<h3 id="Create-List-of-untagged-instances&para;"><a id="2" target="_self" rel="nofollow"></a>Create List of unhealthy instances</h3>
<blockquote>
<p>This action captures the following output: <code>all_unhealthy_instances</code></p>
</blockquote>

In [28]:
all_unhealthy_instances = []
try:
    if unhealthy_instances[0] == False:
        for each_instance in unhealthy_instances[1]:
            all_unhealthy_instances.append(each_instance)
except Exception as e:
    if instance_ids:
        for instance in instance_ids:
            instance_dict = {}
            instance_dict["instance"] = instance
            instance_dict["region"] = region
            all_unhealthy_instances.append(instance_dict)
    else:
        raise Exception(e)

<h3 id="List-Unhealthy-Instances-in-a-Target-Group&para;"><a id="2" target="_self" rel="nofollow"></a>Restart EC2 instances</h3>
<p>Here we will restart all the unhealthy EC2 instances.&nbsp;</p>
<blockquote>
<p>This action takes the following parameters: <code>region, instance_ids</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>None</code></p>
</blockquote>

In [ ]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
import pprint
from beartype import beartype

from beartype import beartype
@beartype
def aws_restart_ec2_instances_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_restart_ec2_instances(handle, instance_ids: List, region: str) -> Dict:
    """aws_restart_instances Restarts instances.

        :type handle: object
        :param handle: Object returned by the task.validate(...) method.

        :type instance_ids: list
        :param instance_ids: List of instance ids.

        :type region: string
        :param region: Region for instance.

        :rtype: Dict with the restarted instances info.
    """

    ec2Client = handle.client('ec2', region_name=region)
    res = ec2Client.reboot_instances(InstanceIds=instance_ids)
    return res


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "instance_ids": "iter.get(\\"instance\\")",
    "region": "iter.get(\\"region\\")"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "all_unhealthy_instances",
    "iter_parameter": ["instance_ids","region"]
    }''')
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_restart_ec2_instances, lego_printer=aws_restart_ec2_instances_printer, hdl=hdl, args=args)

<h3 id="Conclusion&para;&para;">Conclusion</h3>
<p>In this Runbook, we were able to restart all unhealthy EC2 instances in a target group using unSkript's AWS actions. To view the full platform capabilities of unSkript please visit&nbsp;<a href="https://us.app.unskript.io" target="_blank" rel="noopener">us.app.unskript.io</a></p>